# Github Metrics

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ramshreyas/retrospective_grants_metrics/blob/main/github_metrics.ipynb)


---

### Imports & Setup

In [ ]:
# %pip install python-dotenv

In [ ]:
from datetime import datetime
import requests
import json
from IPython.display import display, JSON
from pprint import pprint
from dotenv import load_dotenv
import os
load_dotenv()

GraphQL  Query function

In [ ]:
def execute_graphql_query(query, variables=None, token="your_github_token_here"):
    """
    Executes a GraphQL query to the GitHub API.

    Args:
    query (str): The GraphQL query to execute.
    variables (dict, optional): Variables required for the query. Defaults to None.
    token (str): GitHub API token for authentication.

    Returns:
    dict: The JSON response from the API.
    """
    # Endpoint for GitHub's GraphQL API
    url = 'https://api.github.com/graphql'
    
    # Headers including the authentication token
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    
    # Package the request
    payload = json.dumps({
        'query': query,
        'variables': variables
    })
    
    # Make the HTTP POST request
    response = requests.post(url, headers=headers, data=payload)
    
    # Check if the response was successful
    if response.status_code == 200:
        response_data = response.json()  # Parse JSON response
        display(JSON(response_data))  # Display the response in JSON format for clarity
        return response_data
    else:
        print(f"Query failed with status code {response.status_code}: {response.text}")
        return None

Test

In [ ]:
token = os.getenv("GITHUB_TOKEN")

query = """
query GetRepositoryInfo($owner: String!, $repo: String!) {
    repository(owner: $owner, name: $repo) {
        name
        description
        stargazerCount
    }
}
"""

variables = {
    "owner": "octocat",
    "repo": "Hello-World"
}

# Execute the query
execute_graphql_query(query, variables, token)

HTTP API

In [ ]:
def execute_http_query(url, method='GET', headers=None, data=None, params=None, token=None):
    """
    Executes a generic HTTP request to the GitHub API.
    
    Args:
    url (str): The full URL to the GitHub API endpoint.
    method (stra): The HTTP method to use ('GET', 'POST', etc.).
    headers (dict, optional): Additional HTTP headers as a dictionary.
    data (dict, optional): Data to send with the request (for POST, PUT methods).
    params (dict, optional): URL parameters to append to the request.
    token (str, optional): GitHub API token for authentication.

    Returns:
    tuple: (response content as JSON if available, headers of the response)
    """
    # Initialize headers if not provided
    if headers is None:
        headers = {}
    # Add the authorization token to headers if provided
    if token:
        headers['Authorization'] = f'token {token}'
    headers.setdefault('Accept', 'application/vnd.github.v3+json')

    try:
        # Make the HTTP request
        response = requests.request(method, url, headers=headers, json=data, params=params)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        return response.json(), response.headers
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e.response.status_code} {e.response.reason}")
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError:  # Includes simplejson.decoder.JSONDecodeError
        return response.text, response.headers  # Return plain text if JSON decoding fails

Test

In [ ]:
url = 'https://api.github.com/repos/octocat/Hello-World/issues'  # Example URL to list issues in a repository
response = execute_http_query(url, token=token)
pprint(response)

In [ ]:
owner = "the-candy-shop"
repo = "starksheet-monorepo"

Examples

In [ ]:
# Fetching Contributors
contributors_url = f'https://api.github.com/repos/{owner}/{repo}/contributors'
contributors_response = execute_http_query(contributors_url, token=token)
pprint(contributors_response)

In [ ]:
# Fetching Commit Data
commits_url = f'https://api.github.com/repos/{owner}/{repo}/commits'
commits_response = execute_http_query(commits_url, token=token)
pprint(commits_response)

In [ ]:
# Fetching Issues Data
issues_url = f'https://api.github.com/repos/{owner}/{repo}/issues'
issues_response = execute_http_query(issues_url, token=token)
pprint(issues_response)

In [ ]:
# Fetching Pull Requests
pulls_url = f'https://api.github.com/repos/{owner}/{repo}/pulls'
pulls_response = execute_http_query(pulls_url, token=token)
pprint(pulls_response)

---

## Metrics

Set Org and Repo

In [ ]:
owner = "the-candy-shop"
repo = "starksheet-monorepo"

Fetch repo data

In [ ]:
# Fetching Repository Star Count
stars_url = f'https://api.github.com/repos/{owner}/{repo}'
response = execute_http_query(stars_url, token=token)[0]

Stars

In [ ]:
pprint(response["stargazers_count"])

Forks

In [ ]:
response["forks_count"]

Pull Requests

In [ ]:
def fetch_total_pull_requests(owner, repo, token):
    """
    Fetches the total number of pull requests for a given repository.

    Args:
    owner (str): GitHub repository owner.
    repo (str): GitHub repository name.
    token (str): GitHub API token.

    Returns:
    int: Total number of pull requests.
    """
    base_url = f'https://api.github.com/repos/{owner}/{repo}/pulls'
    params = {'per_page': 100, 'state': 'all'}  # Fetch all PRs, both open and closed
    total_pull_requests = 0
    while True:
        response, headers = execute_http_query(base_url, params=params, token=token)
        total_pull_requests += len(response)
        if 'next' in headers.get('Link', ''):
            next_link = headers['Link']
            # Extract the URL for the next page from the 'Link' header
            next_url = next_link.split(';')[0].strip('<>')
            base_url = next_url  # Update the URL to the next page's URL
        else:
            break

    return total_pull_requests

total_prs = fetch_total_pull_requests(owner, repo, token)
print(f"Total number of pull requests: {total_prs}")

Issues

In [ ]:
query = """
query GetRepositoryInfo($owner: String!, $repo: String!) {
  repository(owner: $owner, name: $repo) {
    all:issues {
      totalCount
    }
    closed:issues(states:CLOSED) {
      totalCount
    }
    open:issues(states:OPEN) {
      totalCount
    }
  }
}
"""

variables = {
    "owner": owner,
    "repo": repo
}

# Execute the query
response = execute_graphql_query(query, variables, token)
print("All: ", response["data"]["repository"]["all"]["totalCount"])
print("Closed: ", response["data"]["repository"]["closed"]["totalCount"])
print("Open: ", response["data"]["repository"]["open"]["totalCount"])

Issue-Resolution turn around time

In [ ]:
def calculate_average_turnaround(data):
    times = []
    for edge in data['data']['repository']['issues']['edges']:
        if edge['node']['closedAt']:
            start = datetime.fromisoformat(edge['node']['createdAt'].replace('Z', '+00:00'))
            end = datetime.fromisoformat(edge['node']['closedAt'].replace('Z', '+00:00'))
            delta = end - start
            times.append(delta.total_seconds())

    average_seconds = sum(times) / len(times) if times else 0
    return average_seconds / 86400  # Convert seconds to days

# Query
query = """
query GetIssueTurnaroundTimes($owner: String!, $repo: String!) {
  repository(owner: $owner, name: $repo) {
    issues(first: 100) {  # Adjust 'first' for pagination
      edges {
        node {
          createdAt
          closedAt
        }
      }
      pageInfo {
        endCursor
        hasNextPage
      }
    }
  }
}
"""

# Execute the query
response_data = execute_graphql_query(query, variables, token)

# Calculate the average turnaround time
average_days = calculate_average_turnaround(response_data)
print(f"Average Turnaround Time: {average_days} days")
